In [1]:
import pandas as pd
import numpy as np
import sys
import multiprocessing
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import lob_for_futures as lobFut
from lob_for_futures import *
# extra imports
from collections import OrderedDict
from itertools import zip_longest
import datetime as dt
from datetime import timedelta

import glob
import fathon
from fathon import fathonUtils as fu

import matplotlib as mpl
from multiprocessing import Pool, freeze_support, Manager

import itertools

colormap = plt.cm.RdBu
plt.style.use(os.path.join(mpl.get_configdir(), 'latexstyle.mplstyle'))
from collections import defaultdict

dataFolder = lobFut.dataFolder
t7 = lobFut.t7folder
expInputFiles = os.path.join(lobFut.augExpertiments, 'ExperimentInputFiles')
HiLoData = os.path.join(lobFut.augExpertiments, 'HiLoData')
plt.style.use(os.path.join('/home/ak/.config/matplotlib', 'latexstyle3.mplstyle'))
symbols = ['RX1', 'FB1', 'JB1', 'G_1', 'FV1', 'TY1', 'TU1', 'DU1', 'YM1', 'XM1', 'US1', 'OE1', 'KE1']

# laptop data folder
laptopDataFolder = os.path.join('/media/ak/OS', 'Data')
expFiles = os.path.join(laptopDataFolder, 'August11th2022Experiments')

destination = os.path.join(laptopDataFolder, 'AthensSoloTripFeb2023')
figuresDestination = '/home/ak/Documents/Papers/StylisedFactsPaper/figures/'
# n: Array of window's sizes used for the computation.
n = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
     39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
     68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]


In [2]:
class CrossCorrel():
    def __init__(self, path, symbols, idx, bar):
        self._idx = idx
        self._symbols = symbols
        self._bar = bar
        self._symbol = self._symbols[self._idx]
        
        self._symbolFilepath = os.path.join(path, str(self._symbol))
        
        self._listOfFiles = os.listdir(self._symbolFilepath)
        
        self._micro_variables = ['arrival_rates', 'gk_vol', 'median_traded_volume', 'micro_price_change']


#         bar = list(set(pkl_dict_keys).difference(set(micro_variables)))[0]  # get out the bar-choice

    def get_data_and_path(self):
    # , fileIdx, symbols, path
        """

        :param symbolIdx: symbol index from list
        :param symbols: list of symbols
        :param bar: bar for information clock
        :param path: which path to read from
        :return: list of files, and path to those files
        """
        #print('data for symbol', self._symbol)
        filesBarSymbol = [f for f in self._listOfFiles if str(self._bar) in f]

        return filesBarSymbol, self._symbolFilepath
    
    def get_all_data_from_file(self, fileIdx):
        """
        fileIdx = index file to get unpickled
        """
        filesBarSymbol, symbolFilepath = self.get_data_and_path()
        fileToGet = os.path.join(symbolFilepath , filesBarSymbol[fileIdx])
        return pd.read_pickle(fileToGet)
    
    def get_microvar_data(self, fileIdx,var): 
        pklDict= self.get_all_data_from_file(fileIdx)
        return pklDict[str(var)]
    
    def get_all_microvar_data(self, fileIdx):
        pkl_dict= self.get_all_data_from_file(fileIdx)
        pkl_dict_keys = sorted(list(pkl_dict.keys())) # get out all the keys- but its really bars
        
        #----micro structure vars-----#
        
        gk_vol = pkl_dict['gk_vol']  # get the vol
        median_traded_volume = pkl_dict['median_traded_volume']  # get the volume
        arrival_rates = pkl_dict['arrival_rates']  # get arrival rates
        micro_price_change = pkl_dict['micro_price_change']  # get micro price change
         
        # ---- start getting all the mfdfa variables ----#
        
        tau_dict = pkl_dict[str(self._bar)]['tau']  # tau
        alpha = pkl_dict[str(self._bar)]['alpha']  # alpha
        mfSpect = pkl_dict[str(self._bar)]['mfSpect']  # mfSpect
        n_F_dict = pkl_dict[str(self._bar)]['n_F']  # n_F
        list_H_dict = pkl_dict[str(self._bar)]['list_H']  # list_H
        list_H_intercept = pkl_dict[str(self._bar)]['list_H_intercept']  # intercept
        
        return  gk_vol, median_traded_volume, arrival_rates, micro_price_change, tau_dict, alpha, mfSpect, n_F_dict, list_H_dict, list_H_intercept
    
        
        
#

In [3]:
symbols = ['RX1', 'FB1', 'JB1', 'G_1', 'FV1', 'TY1', 'TU1', 'DU1', 'YM1', 'XM1', 'US1', 'OE1', 'KE1']
micro_variables = ['arrival_rates', 'gk_vol', 'median_traded_volume', 'micro_price_change']
# self, path, symbols, idx, bar, use_var
cc2 = CrossCorrel(laptopDataFolder, symbols, 5, 'tick')

In [8]:
######################################################### 

In [9]:
######################################################### 

In [4]:
files, filesPath = cc2.get_data_and_path()



In [5]:
gk_vol, median_traded_volume, arrival_rates, micro_price_change, tau_dict, alpha, mfSpect, n_F_dict, list_H_dict, list_H_intercept = cc2.get_all_microvar_data(fileIdx = 1)

In [6]:
range_to_use = range(0,10)
arrivals_list = {f:cc2.get_microvar_data(fileIdx= f, var='arrival_rates') for f in range_to_use }
median_volumes_list = {f:cc2.get_microvar_data(fileIdx= f, var='median_traded_volume') for f in range_to_use }


In [7]:
# get all the data fast!

In [8]:
arrivalsDF = pd.DataFrame(arrivals_list).fillna(0)
medianVolumesDF = pd.DataFrame(median_volumes_list).fillna(0)


In [ ]:
medianVolumesDF

In [9]:
import glob
import fathon
from fathon import fathonUtils as fu
winSizes = fu.linRangeByStep(10, 1000, step=20)
polOrd = 1
import time


In [10]:
idx =1 
a =arrivalsDF[idx]
b= medianVolumesDF[idx]

a = fu.toAggregated(a)
b = fu.toAggregated(b)
pydcca = fathon.DCCA(a, b)
n, F = pydcca.computeFlucVec(winSizes, polOrd=polOrd)
H, H_intercept = pydcca.fitFlucVec()
####
# limits_list = np.array([[10,120], [220,1000]], dtype=int)
# list_H, list_H_intercept = pydcca.multiFitFlucVec(limits_list)

n, rho = pydcca.computeRho(winSizes, polOrd=polOrd)

In [17]:
def compute_h_h_dcca(var_a, var_b, idx):
    """
    var_a = first variable for the dCCa rho
    var_b = second variable for the dCCa rho
    idx = index in for which "day to pick"
    """
    a =var_a[idx]
    b= var_b[idx]

    a = fu.toAggregated(a)
    b = fu.toAggregated(b)
#     pydcca = fathon.DCCA(a, b)
# #     n, F = pydcca.computeFlucVec(winSizes, polOrd=polOrd)
#     H, H_intercept = pydcca.fitFlucVec()

    return a, b

In [21]:
a, b =compute_h_h_dcca(arrivalsDF, medianVolumesDF, idx)

In [24]:
pydcca = fathon.DCCA(a, b)
n, F = pydcca.computeFlucVec(winSizes, polOrd=polOrd)
H, H_intercept = pydcca.fitFlucVec()

In [28]:
H_intercept

0.9016279378567426

In [ ]:


rhoDict =dict()
n =[]
for idx in range(0,5):
    n, rhoDict[idx] = compute_n_Rho(arrivalsDF, medianVolumesDF, idx)

In [ ]:
rhoDF = pd.DataFrame.from_dict(rhoDict)


In [ ]:
plt.rcParams['font.sans-serif'] = 'Verdana'#'Helvetica'
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.8
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'

fig, ax = plt.subplots(figsize=(12,8))

ax.set_facecolor('xkcd:white')
plt.plot(np.log(n), np.log(F), 'bo',alpha=0.6,markersize=9, label =" ")
plt.plot(np.log(n), H_intercept+H*np.log(n), 'k:',color='r', linewidth=3 ,label='H = {:.2f}'.format(H))
plt.xlabel('ln(n)', fontsize=24)
plt.ylabel('ln(F(n))', fontsize=24)
plt.yticks(fontsize=16)

plt.xticks(fontsize=16)

plt.legend(loc=0, fontsize=16)

# file_name = str(symbol)+'_'+str(bar)+'_lnFn_n.png'
# _=plt.savefig(os.path.join(figuresDestination,file_name),dpi=300, bbox_inches="tight")

_=plt.show()

In [ ]:
plt.rcParams['font.sans-serif'] = 'Verdana'#'Helvetica'
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.8
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'

fig, ax = plt.subplots(figsize=(12,8))
plt.plot(n, rhoDF[1], 'ro', alpha =1.5, markersize = 10,color='darkgreen')
plt.plot(n, rhoDF[3], 'ro', alpha =1.5, markersize = 10,color='darkred')
plt.plot(n, rhoDF[2], 'ro', alpha =1.5, markersize = 10,color='grey')
plt.plot(n, rhoDF[4], 'ro', alpha =1.5, markersize = 10,color='lightblue')
plt.plot(n, rhoDF[0], 'ro', alpha =1.5, markersize = 10,color='lightblue')

plt.ylim(-1, 1)
plt.xlabel('win-size', fontsize=24)
plt.ylabel('$\\rho_{DCCA}$', fontsize=24)
plt.yticks(fontsize=16)
ax.set_facecolor('xkcd:white')
plt.xticks(fontsize=16)

_=plt.xlim(0, 1000)
_=plt.ylim(-0.2, 0.5)

In [ ]:
_=plt.hist(rhoDF[1], bins =100)
_=plt.hist(rhoDF[3], bins =100)
_=plt.hist(rhoDF[4], bins =200)



In [ ]:
import multiprocessing
def printVal(value):
    return ("hello "*(value))
pool_obj = multiprocessing.Pool()
ans = pool_obj.map(printVal,range(0,4))
print(ans)
pool_obj.close()

In [ ]:
def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

In [31]:
hurst_files =[f for f in os.listdir(expInputFiles) if str('Hurst') in f]

In [33]:
file = os.path.join(expInputFiles, hurst_files[2])
pd.read_pickle(file)['H']

0.781737570160062

In [10]:
#################### imports ######################

In [15]:
rhoFiles = sorted([f for f in os.listdir(expInputFiles) if str('rho') in f])
hurstFiles = sorted([f for f in os.listdir(expInputFiles) if str('Hurst') in f])

In [17]:
filepath = os.path.join(expInputFiles, hurstFiles[1])